# Setup

In [1]:
import time
import boto3
import logging
import ipywidgets as widgets
import uuid

from agent import create_agent_role, create_lambda_role
from agent import create_dynamodb, create_lambda, invoke_agent_helper

## Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

('us-east-1', '161242380044')

# Setting up Agent's information

In [2]:
suffix = f"{region}-{account_id}"
agent_name = 'lol-coach-agent'
agent_bedrock_allow_policy_name = f"{agent_name}-ba"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'

agent_description = "Agent in charge of managing player data and doing web searches for League of Legends related information."
agent_instruction = """
You are an expert League of Legends coach agent, helping players retrieve game data, create a new League of Legends player profile, and find League of Legends related information through web searches.
"""

# Select Foundation Model

In [3]:
agent_foundation_model_selector = widgets.Dropdown(
    options=[
        ('Claude 3 Sonnet', 'anthropic.claude-3-sonnet-20240229-v1:0'),
        ('Claude 3 Haiku', 'anthropic.claude-3-haiku-20240307-v1:0')
    ],
    value='anthropic.claude-3-sonnet-20240229-v1:0',
    description='FM:',
    disabled=False,
)
print(agent_foundation_model_selector)

agent_foundation_model = agent_foundation_model_selector.value
print(agent_foundation_model)

Dropdown(description='FM:', options=(('Claude 3 Sonnet', 'anthropic.claude-3-sonnet-20240229-v1:0'), ('Claude 3 Haiku', 'anthropic.claude-3-haiku-20240307-v1:0')), value='anthropic.claude-3-sonnet-20240229-v1:0')
anthropic.claude-3-sonnet-20240229-v1:0


# Creating Agent

In [4]:
agent_role = create_agent_role(agent_name, agent_foundation_model)
print(agent_role)

response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
print(response)

agent_id = response['agent']['agentId']
print("The agent id is:",agent_id)

{'Role': {'Path': '/', 'RoleName': 'AmazonBedrockExecutionRoleForAgents_lol-coach-agent', 'RoleId': 'AROASLCWLO4GENFDXKKU4', 'Arn': 'arn:aws:iam::161242380044:role/AmazonBedrockExecutionRoleForAgents_lol-coach-agent', 'CreateDate': datetime.datetime(2025, 10, 11, 7, 32, 46, tzinfo=tzutc()), 'AssumeRolePolicyDocument': {'Version': '2012-10-17', 'Statement': [{'Effect': 'Allow', 'Principal': {'Service': 'bedrock.amazonaws.com'}, 'Action': 'sts:AssumeRole'}]}, 'MaxSessionDuration': 3600, 'RoleLastUsed': {'LastUsedDate': datetime.datetime(2025, 10, 11, 16, 38, 24, tzinfo=tzutc()), 'Region': 'us-east-1'}}, 'ResponseMetadata': {'RequestId': '25e512ea-cadb-4c90-a3fa-a8e82486c620', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 11 Oct 2025 16:49:37 GMT', 'x-amzn-requestid': '25e512ea-cadb-4c90-a3fa-a8e82486c620', 'content-type': 'text/xml', 'content-length': '1006'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': '6cebcaad-4817-4842-9ae6-10ce86af47ae', 'HTTPStatusCode': 202, 'HT

# Creating DynamoDB Table

In [5]:
table_name = 'player_data'
# create_dynamodb(table_name)

# Creating Lambda Function for Data retrieval Action Group

In [6]:
%%writefile lambda_function.py
import json
import uuid
import boto3

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('player_data')

def get_named_parameter(event, name):
    """
    Get a parameter from the lambda event
    """
    return next(item for item in event['parameters'] if item['name'] == name)['value']

def get_player_data(player_id):
    """
    Retrieve game data of a specific player
    
    Args:
        player_id (string): The unique identifier for the player.
    """
    try:
        response = table.get_item(Key={'player_id': player_id})
        if 'Item' in response:
            return response['Item']
        else:
            return {'message': f'No data found with ID {player_id}'}
    except Exception as e:
        return {'error': str(e)}
    
def create_lol_player(player_name, rank, region, main_role, favorite_champion):
    """
    Create a new League of Legends player profile.
    
    Args:
        player_name (string): The in-game name of the player.
        rank (string): The current rank of the player (e.g. 'Gold IV', 'Diamond I').
        region (string): The server region (e.g. 'NA', 'EUW', 'SG').
        main_role (string): The player’s main role (e.g. 'Top', 'Jungle', 'Mid', 'ADC', 'Support').
        favorite_champion (string): The champion the player uses most frequently.
    """
    try:
        player_id = str(uuid.uuid4())[:8]
        table.put_item(
            Item={
                'player_id': player_id,
                'player_name': player_name,
                'rank': rank,
                'region': region,
                'main_role': main_role,
                'favorite_champion': favorite_champion
            }
        )
        return {'player_id': player_id}
    except Exception as e:
        return {'error': str(e)}
    
def lambda_handler(event, context):
    # get the action group used during the invocation of the lambda function
    actionGroup = event.get('actionGroup', '')
    
    # name of the function that should be invoked
    function = event.get('function', '')
    
    # parameters to invoke function with
    parameters = event.get('parameters', [])

    if function == 'get_player_data':
        player_id = get_named_parameter(event, "player_id")
        if player_id:
            response = str(get_player_data(player_id))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': 'Missing player_id parameter'}}
    
    elif function == 'create_lol_player':
        player_name = get_named_parameter(event, "player_name")
        rank = get_named_parameter(event, "rank")
        region = get_named_parameter(event, "region")
        main_role = get_named_parameter(event, "main_role")
        favorite_champion = get_named_parameter(event, "favorite_champion")
        
        if all([player_name, rank, region, main_role, favorite_champion]):
            response = str(create_lol_player(player_name, rank, region, main_role, favorite_champion))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': 'Missing required parameters'}}

    else:
        responseBody = {'TEXT': {'body': 'Invalid function'}}

    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }
    }

    function_response = {'response': action_response, 'messageVersion': event['messageVersion']}
    print("Response: {}".format(function_response))

    return function_response

Overwriting lambda_function.py


In [7]:
lambda_iam_role = create_lambda_role(agent_name, table_name)
data_retrieval_lambda_function_name = f'{agent_name}-data-retrieval-lambda'
data_retrieval_lambda_function = create_lambda(data_retrieval_lambda_function_name, lambda_iam_role)

# Create Data retrieval Action Group

In [8]:
agent_functions_for_data_retrieval = [
    {
        'name': 'get_player_data',
        'description': 'Retrieve game data of a specific player',
        'parameters': {
            "player_id": {
                "description": "The unique identifier for the player",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'create_lol_player',
        'description': 'Create a new League of Legends player profile',
        'parameters': {
            "player_name": {
                "description": "The in-game name of the player",
                "required": True,
                "type": "string"
            },
            "rank": {
                "description": "The current rank of the player (e.g. 'Gold IV', 'Diamond I')",
                "required": True,
                "type": "string"
            },
            "region": {
                "description": "The server region (e.g. 'NA', 'EUW', 'SG')",
                "required": True,
                "type": "string"
            },
            "main_role": {
                "description": "The player’s main role (e.g. 'Top', 'Jungle', 'Mid', 'ADC', 'Support')",
                "required": True,
                "type": "string"
            },
            "favorite_champion": {
                "description": "The champion the player uses most frequently",
                "required": True,
                "type": "string"
            }
        }
    }
]

# Pause to make sure agent is created
time.sleep(30)

# Now, we can configure and create an action group here:
agent_data_retrieval_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': data_retrieval_lambda_function['FunctionArn']
    },
    actionGroupName='DataRetrievalActionGroup',
    functionSchema={
        'functions': agent_functions_for_data_retrieval
    },
    description='Actions for retrieving game data of a specific player and creating a new League of Legends player profile.'
)

print(agent_data_retrieval_action_group_response)

{'ResponseMetadata': {'RequestId': 'eb1644a9-6482-426a-a8b3-af1e023460fc', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 11 Oct 2025 16:50:12 GMT', 'content-type': 'application/json', 'content-length': '1471', 'connection': 'keep-alive', 'x-amzn-requestid': 'eb1644a9-6482-426a-a8b3-af1e023460fc', 'x-amz-apigw-id': 'SSqepGGGIAMEBDQ=', 'x-amzn-trace-id': 'Root=1-68ea8ac3-6a32ee150b510e9e6255f643'}, 'RetryAttempts': 0}, 'agentActionGroup': {'agentId': 'UFSZYQKWFU', 'agentVersion': 'DRAFT', 'actionGroupId': '3SS4CGJGS1', 'actionGroupName': 'DataRetrievalActionGroup', 'description': 'Actions for retrieving game data of a specific player and creating a new League of Legends player profile.', 'createdAt': datetime.datetime(2025, 10, 11, 16, 50, 11, 884987, tzinfo=tzutc()), 'updatedAt': datetime.datetime(2025, 10, 11, 16, 50, 11, 884987, tzinfo=tzutc()), 'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-east-1:161242380044:function:lol-coach-agent-data-retrieval-lambda'}, 'functionSc

## Allowing bedrock to invoke lambda function for Data retrieval Action Group

In [9]:
# Create allow to invoke permission on lambda
data_retrieval_lambda_client = boto3.client('lambda')
try:
    response = data_retrieval_lambda_client.add_permission(
        FunctionName=data_retrieval_lambda_function_name,
        StatementId=f'allow_bedrock_{agent_id}',
        Action='lambda:InvokeFunction',
        Principal='bedrock.amazonaws.com',
        SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
    )
    print(response)
except Exception as e:
    print(e)

{'ResponseMetadata': {'RequestId': 'ebd82a34-c54b-4c2e-b3bc-e2b731bd0b3c', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Sat, 11 Oct 2025 16:50:12 GMT', 'content-type': 'application/json', 'content-length': '376', 'connection': 'keep-alive', 'x-amzn-requestid': 'ebd82a34-c54b-4c2e-b3bc-e2b731bd0b3c'}, 'RetryAttempts': 0}, 'Statement': '{"Sid":"allow_bedrock_UFSZYQKWFU","Effect":"Allow","Principal":{"Service":"bedrock.amazonaws.com"},"Action":"lambda:InvokeFunction","Resource":"arn:aws:lambda:us-east-1:161242380044:function:lol-coach-agent-data-retrieval-lambda","Condition":{"ArnLike":{"AWS:SourceArn":"arn:aws:bedrock:us-east-1:161242380044:agent/UFSZYQKWFU"}}}'}


# Creating Lambda Function for Web search Action Group

In [10]:
%%writefile lambda_function.py
import http.client
import json
import logging
import os
import urllib.request

log_level = os.environ.get("LOG_LEVEL", "INFO").strip().upper()
logging.basicConfig(format="[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(log_level)

AWS_REGION = os.environ.get("AWS_REGION", "us-east-1")
ACTION_GROUP_NAME = "WebSearchActionGroup"
FUNCTION_NAMES = ["tavily-ai-search", "google-search"]


SERPER_API_KEY = os.environ.get("SERPER_API_KEY")
TAVILY_API_KEY = os.environ.get("TAVILY_API_KEY")


def extract_search_params(action_group, function, parameters):
    if action_group != ACTION_GROUP_NAME:
        logger.error(f"unexpected name '{action_group}'; expected valid action group name '{ACTION_GROUP_NAME}'")
        return None, None

    if function not in FUNCTION_NAMES:
        logger.error(f"unexpected function name '{function}'; valid function names are'{FUNCTION_NAMES}'")
        return None, None

    search_query = next(
        (param["value"] for param in parameters if param["name"] == "search_query"),
        None,
    )

    target_website = next(
        (param["value"] for param in parameters if param["name"] == "target_website"),
        None,
    )

    logger.debug(f"extract_search_params: {search_query=} {target_website=}")

    return search_query, target_website


def google_search(search_query: str, target_website: str = "") -> str:
    query = search_query
    if target_website:
        query += f" site:{target_website}"

    conn = http.client.HTTPSConnection("google.serper.dev")
    payload = json.dumps({"q": query})
    headers = {"X-API-KEY": SERPER_API_KEY, "Content-Type": "application/json"}

    search_type = "news"  # "news", "search",
    conn.request("POST", f"/{search_type}", payload, headers)
    res = conn.getresponse()
    data = res.read()

    return data.decode("utf-8")


def tavily_ai_search(search_query: str, target_website: str = "") -> str:
    logger.info(f"executing Tavily AI search with {search_query=}")

    base_url = "https://api.tavily.com/search"
    headers = {"Content-Type": "application/json", "Accept": "application/json"}
    payload = {
        "api_key": TAVILY_API_KEY,
        "query": search_query,
        "search_depth": "advanced",
        "include_images": False,
        "include_answer": False,
        "include_raw_content": False,
        "max_results": 3,
        "include_domains": [target_website] if target_website else [],
        "exclude_domains": [],
    }

    data = json.dumps(payload).encode("utf-8")
    request = urllib.request.Request(base_url, data=data, headers=headers)  # nosec: B310 fixed url we want to open

    try:
        response = urllib.request.urlopen(request)  # nosec: B310 fixed url we want to open
        response_data: str = response.read().decode("utf-8")
        logger.debug(f"response from Tavily AI search {response_data=}")
        return response_data
    except urllib.error.HTTPError as e:
        logger.error(f"failed to retrieve search results from Tavily AI Search, error: {e.code}")

    return ""


def lambda_handler(event, _):  # type: ignore
    logging.debug(f"lambda_handler {event=}")

    action_group = event["actionGroup"]
    function = event["function"]
    parameters = event.get("parameters", [])

    logger.info(f"lambda_handler: {action_group=} {function=}")

    search_query, target_website = extract_search_params(action_group, function, parameters)

    search_results: str = ""
    if function == "tavily-ai-search":
        search_results = tavily_ai_search(search_query, target_website)
    elif function == "google-search":
        search_results = google_search(search_query, target_website)

    logger.debug(f"query results {search_results=}")

    # Prepare the response
    function_response_body = {"TEXT": {"body": f"Here are the top search results for the query '{search_query}': {search_results} "}}

    action_response = {
        "actionGroup": action_group,
        "function": function,
        "functionResponse": {"responseBody": function_response_body},
    }

    response = {"response": action_response, "messageVersion": event["messageVersion"]}

    logger.debug(f"lambda_handler: {response=}")

    return response

Overwriting lambda_function.py


In [11]:
web_search_lambda_function_name = f'{agent_name}-web-search-lambda'
web_search_lambda_function = create_lambda(web_search_lambda_function_name, lambda_iam_role)

# Create Web search Action Group

In [12]:
agent_functions_for_web_search = [
    {
        'name': 'tavily-ai-search',
        'description': """
            To retrieve information via the internet
            or for topics that the LLM does not know about and
            intense research is needed.
        """,
        'parameters': {
            "search_query": {
                "description": "The search query for the Tavily web search.",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'google-search',
        'description': 'For targeted news, like "What are the latest League of Legends updates?" or similar.',
        'parameters': {
            "search_query": {
                "description": "The search query for the Google web search.",
                "required": True,
                "type": "string"
            }
        }
    }
]

# Pause to make sure agent is created
time.sleep(30)

# Now, we can configure and create an action group here:
agent_web_search_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': web_search_lambda_function['FunctionArn']
    },
    actionGroupName='WebSearchActionGroup',
    functionSchema={
        'functions': agent_functions_for_web_search
    },
    description='Actions for web searching League of Legends related information.'
)

print(agent_web_search_action_group_response)

{'ResponseMetadata': {'RequestId': '46a172b6-5493-45f4-b2ee-958009b9ff20', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 11 Oct 2025 16:50:44 GMT', 'content-type': 'application/json', 'content-length': '1143', 'connection': 'keep-alive', 'x-amzn-requestid': '46a172b6-5493-45f4-b2ee-958009b9ff20', 'x-amz-apigw-id': 'SSqjqEURoAMEgNg=', 'x-amzn-trace-id': 'Root=1-68ea8ae3-0cfd973957c2c9874a9e24ca'}, 'RetryAttempts': 0}, 'agentActionGroup': {'agentId': 'UFSZYQKWFU', 'agentVersion': 'DRAFT', 'actionGroupId': 'OFXYDCOV2Y', 'actionGroupName': 'WebSearchActionGroup', 'description': 'Actions for web searching League of Legends related information.', 'createdAt': datetime.datetime(2025, 10, 11, 16, 50, 44, 30601, tzinfo=tzutc()), 'updatedAt': datetime.datetime(2025, 10, 11, 16, 50, 44, 30601, tzinfo=tzutc()), 'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-east-1:161242380044:function:lol-coach-agent-web-search-lambda'}, 'functionSchema': {'functions': [{'name': 'tavily-ai-search', '

## Allowing bedrock to invoke lambda function for Web search Action Group

In [13]:
# Create allow to invoke permission on lambda
web_search_lambda_client = boto3.client('lambda')
try:
    response = web_search_lambda_client.add_permission(
        FunctionName=web_search_lambda_function_name,
        StatementId=f'allow_bedrock_{agent_id}',
        Action='lambda:InvokeFunction',
        Principal='bedrock.amazonaws.com',
        SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
    )
    print(response)
except Exception as e:
    print(e)

{'ResponseMetadata': {'RequestId': '5eab6265-3c97-441b-9693-e5fb23c362cf', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Sat, 11 Oct 2025 16:50:45 GMT', 'content-type': 'application/json', 'content-length': '372', 'connection': 'keep-alive', 'x-amzn-requestid': '5eab6265-3c97-441b-9693-e5fb23c362cf'}, 'RetryAttempts': 0}, 'Statement': '{"Sid":"allow_bedrock_UFSZYQKWFU","Effect":"Allow","Principal":{"Service":"bedrock.amazonaws.com"},"Action":"lambda:InvokeFunction","Resource":"arn:aws:lambda:us-east-1:161242380044:function:lol-coach-agent-web-search-lambda","Condition":{"ArnLike":{"AWS:SourceArn":"arn:aws:bedrock:us-east-1:161242380044:agent/UFSZYQKWFU"}}}'}


## Preparing agent

In [14]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)
# Pause to make sure agent is prepared
time.sleep(30)

{'ResponseMetadata': {'RequestId': 'e7ad2a7e-9319-4ddc-9347-6548683b3f81', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Sat, 11 Oct 2025 16:50:45 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': 'e7ad2a7e-9319-4ddc-9347-6548683b3f81', 'x-amz-apigw-id': 'SSqj6EMYIAMEguA=', 'x-amzn-trace-id': 'Root=1-68ea8ae5-1b0256635f5259b5666bcb86'}, 'RetryAttempts': 0}, 'agentId': 'UFSZYQKWFU', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2025, 10, 11, 16, 50, 45, 723184, tzinfo=tzutc())}


# Invoking Agent

In [15]:
alias_id = 'TSTALIASID'

In [16]:
"""
%%time
session_id:str = str(uuid.uuid1())
query = "I want to create a new League of Legends player profile for a user named “ShadowFox”, ranked Diamond I, playing in the Singapore server, main role Mid, and favorite champion Yasuo."
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)
"""

'\n%%time\nsession_id:str = str(uuid.uuid1())\nquery = "I want to create a new League of Legends player profile for a user named “ShadowFox”, ranked Diamond I, playing in the Singapore server, main role Mid, and favorite champion Yasuo."\nresponse = invoke_agent_helper(query, session_id, agent_id, alias_id)\nprint(response)\n'

In [17]:
%%time
session_id:str = str(uuid.uuid1())
query = "I want to retrieve the favorite champion for the player with the id '5130f179'."
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

The favorite champion for the player with ID '5130f179' is Yasuo.
CPU times: user 27.9 ms, sys: 4.75 ms, total: 32.6 ms
Wall time: 5.54 s


In [19]:
%%time
session_id:str = str(uuid.uuid1())
query = "What are the latest updates in League of Legends?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

The latest League of Legends updates include:

- Patch 15.20 with balance changes, new skins, and Worlds 2025 preparations
- New "2XKO" themed content for Teamfight Tactics
- Ongoing patches like 25.19 and 25.21 making balance adjustments for Worlds 2025
- Previews of major changes coming in Season 3, set in the year 2025

The patches are focused on getting League of Legends ready for the upcoming 2025 World Championship tournament, while also introducing new gameplay content and cosmetics.
CPU times: user 29.3 ms, sys: 11.9 ms, total: 41.2 ms
Wall time: 23 s


In [20]:
%store agent_role

Stored 'agent_role' (dict)
